In [1]:
# Skriptas Zive įrašų talpykloje kiekvienam pacientui atrenka iki n_max įrašų su daugiausiai
# turinčiais ekstrasistolių, sudaro jų sąrašą ir patalpina į atskirą folderį.
# 
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26

# Planas:
# sukuriame tuščią dataframe df_list_selected, iš kurio darysime naują list.json
# Nuskaitome iš bendros talpyklos list.json
# pasidarom sąrašą userId:recordId1, recordId2,.... 
# ciklas per visus pacientus userId:
#   suformuojame sąrašą recordId: ekstrasistolių skaičius ir surušiuojame pagal ekstrasistolių
#   skaičių mažėjančia tvarka
#   Surikiavus įrašus, įrašus atrenkame tokiu būdu:
# 		a. Atrenkame pirmiausiai tuos įrašus, kurie turi daugiausiai ekstrasistolių. 
#       Jei visi įrašai turi ekstrasistolių, tai atrenkame pirmus max_count (iš surikiuotų įrašų pagal ekstrasistolių kiekį).
#       Jei ne visi įrašai turi ekstrasistoles, tai atrenkame, tik tuos įrašus, kurie turi ekstrasistoles
#       (bet ne daugiau max_count įrašų), ir jei įrašų atrinkta ne daugiau max_count-1, pridedame dar vieną įrašą be ekstrasistolių.
# 		b. Jei pacientas ekstrasistolių neturi, tai atrenkame ne daugiau vieno (pirmo) to paciento įrašo

import os, shutil, sys
from pathlib import Path
import pandas as pd
import numpy as np

import json
from zive_util_vu import create_dir

import warnings
# warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'DUOM_2022_RUDUO_3'

# Nuoroda į aplanką su EKG duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder, 'records')

# Nuoroda į aplanką su atrinktų EKG duomenų rinkiniu
db_path_selected = Path(Duomenu_aplankas, db_folder, 'records_selected')

# Atrenkamų iš vieno paciento įrašų max skaičius
max_count = 500


# ////////////////////////////////////////////////////////////////////////////////////

print("\nSkriptas zive įrašų atrankai klasifikatorių mokymui, vertinimui, testavimui\n")

# Sukūriame aplanką EKG sekų įrašymui
create_dir(db_path_selected)

# paruošiame dataframe atrinktų įrašų sąrašui

# df_list_selected = pd.DataFrame({
#                         'file_name': pd.Series(dtype='str'),
#                         'userId': pd.Series(dtype='str'),
#                         'recordingId': pd.Series(dtype='str'),
#                         'N': pd.Series(dtype='int'),
#                         'S': pd.Series(dtype='int'),
#                         'V': pd.Series(dtype='int'),
#                         'U': pd.Series(dtype='int'),
#                         'incl': pd.Series(dtype='int'), 
#                         'flag': pd.Series(dtype='int'),
#                         'recorded_at':pd.Series(dtype='str'), 
#                         'E': pd.Series(dtype='int')
#                         })

df_list_selected = pd.DataFrame()                        

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns", 18)
pd.set_option('display.width', 1000)

# Nuskaitome Zive įrašų talpykloje laikomų įrašų sąrašą
file_path = Path(db_path, 'list.json')
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())

df_list = pd.json_normalize(data, record_path =['data'])

# Suformuojame sąraše naują kolonėlę su ekstrasistolių kiekiu
df_list['E'] = df_list['S'] + df_list['V']
# print(df_list.head(20))

# Sugrupuojame įrašus į to paties paciento grupes
df_grouped = df_list.groupby(['userId'])
# print(df_grouped)

# Išrenkame įrašus su didžiausiu kiekiu ekstrasistolių, formuojame sąrašą,
# perrašome įrašus į atskirą folderį

for name, group in df_grouped:
    sorted_group = group.sort_values(by='E', ascending=False)
    records_count = len(sorted_group)
    if (records_count > max_count):
        records_count = max_count
    # print(f'{max_count=}')
             
    count = 0    
    for idx, df_col in sorted_group.iterrows():
    # Einame per visus to paties paciento 'name' įrašus, esančius sorted_group sąraše
        df_row = pd.Series(df_col).to_frame().T
        cond = (df_row['E'] == 0).any()
        # Jei paciento 'name' įrašas iš sąrašo neturi ekstrasistolių, tai jis įtraukiamas į atrinktų
        # įrašų sąrašą, ciklas per to paciento įrašus nutraukiamas. Jei tai pirmas įrašas iš sąrašo
        # 'sorted_group", tada atrinktų įrašų sąraše bus tik šis vienas įrašas šiam pacientui. Jei prieš tai
        # buvo surasti max_count įrašai su ekstrasistolėmis, tai bus paskutinis šio paciento įrašas. 
        if cond:
            df_list_selected = pd.concat([df_list_selected, df_row], axis = 0)
            break 
        else:
            # Jei pirmas įrašas turėjo ekstrasistolių, toliau tesiamas ciklas per paciento 'name' įrašus
            count +=1
            cond = (df_row['E'] != 0).any()
            # Jei paciento 'name' įrašas turi ekstrasistolių, jis įtraukiamas į atrinktų įrašų sąrašą,
            # jei jau įtrauktų skaičius neviršija užduotos ribos max_count 
            if  (count <= max_count and cond):
                df_list_selected = pd.concat([df_list_selected, df_row], axis = 0)
            else:
                cond = (df_row['E'] == 0).any()
                # Jei paciento 'name' įrašas iš sąrašo neturi ekstrasistolių, tai jis įtraukiamas į atrinktų
                # įrašų sąrašą, kaip paskutinis ir ciklas per to paciento įrašus nutraukiamas.
                if (cond):
                    df_list_selected = pd.concat([df_list_selected, df_row], axis = 0)
                break

# Sutvarkome kintamųjų tipus
df_list_selected[["N", "S", "V", "U", "incl", "flag", "E"]] = df_list_selected[["N", "S", "V", "U", "incl", "flag", "E"]].apply(pd.to_numeric)
# print(df_list_selected.info())


# Perrašome atrinktus įrašus į diską
for idx, row in df_list_selected.iterrows():
    file_name = row['file_name']
    src = Path(db_path, file_name)
    dst = Path(db_path_selected, file_name)
    shutil.copy2(src, dst)
    src = Path(db_path, file_name + '.json')
    dst = Path(db_path_selected, file_name + '.json')
    shutil.copy2(src, dst)
# https://www.geeksforgeeks.org/python-shutil-copy2-method/#:~:text=shutil.copy
# 2%20%28%29%20method%20in%20Python%20is%20used%20to,destination%20can%20be%20a%20file%20or%20a%20directory.


# Apskaičiuojame ir išvedame statistiką
df_sum = df_list_selected.groupby('userId').sum()

# https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/
counts = df_list_selected['userId'].value_counts()
print(f'\nViso pacientų: {len(counts)}  EKG įrašų: {len(df_list_selected)}')

print(f'\nAtrinktų įrašų skaičiaus pasiskirstymas per pacientus (surūšiuotas)')
print(f'userId                      Įrašų skaičius')
print(counts)

counts = counts.rename("counts")
frames = [df_sum, counts]
result = pd.concat(frames, axis = 1)
result.index.rename ('userID', inplace= True)
result.drop(labels=['incl'], axis=1, inplace=True)
result = result.sort_values(by='counts', ascending=False)
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/

print("\nAtrinktų pacientų sąrašas (surušiuotas pagal paciento įrašų skaičių - counts)")
print(result)

# Susumuojamos anotacijos
suma = result.iloc[:,0:7].sum()
print('\n',' '*18, 'sum: ',str(suma.tolist())[1:-1])

selected_columns = result[['E']]
result_new = selected_columns.copy()
# print(result_new)

sorted_group = result_new.sort_values(by='E', ascending=False)
print("\nAtrinktų ir surūšiuotų pagal bendrą ekstrasistolių kiekį E pacientų sąrašas")
print(sorted_group)


# Išplėstinė statistika
print("\nIšplėstinis pacientų sąrašas, surūšiuotas pagal įrašų skaičių ir ")
print("šių pacientų EKG įrašų sąrašai, surušiuoti pagal bendrą ekstrasistolių kiekį E")
grouped  = df_list_selected.groupby('userId')
lst_grouped  = df_list_selected.groupby('userId').size().sort_values(ascending=False)
userIds = lst_grouped.index

for userId in userIds:
    group = grouped.get_group(userId)
    print(f"\nuserId: {userId}  EKG įrašų: {len(group)} " )
    tit1 = f"{'recordingId':>6} {'file_name':>31} {'N':>10} {'E':>8}"
    tit2 =f" {'S':>9}   {'V':>8}   {'U':>8} " 
    print(tit1+tit2)
    for idx, row in group.iterrows():
        str1 =f"{(row['recordingId']):>6} {str(row['file_name']):>20} {int(row['N']):>8} {int(row['E']):>8} " 
        str2 =f" {int(row['S']):>8}   {int(row['V']):>8}   {int(row['U']):>8} " 
        print(str1+str2)

# https://www.statology.org/pandas-groupby-sort/
# print("\ngrouped")
# print(grouped)


# Įrašome naujai suformuotą sąrašą į diską

# Prieš įrašant į diską, išmetame stulpelį 'E'
df_list_selected.drop(labels=['E'], axis=1, inplace=True)

file_path = Path(db_path_selected,'list.json')       
df_list_selected.to_json(file_path, orient = 'table', index=False)
print(f'\nFailų sąrašas įrašytas:  {file_path}')


OS in my system :  linux

Skriptas zive įrašų atrankai klasifikatorių mokymui, vertinimui, testavimui

Directory '/home/kesju/DI/DUOM_2022_RUDUO_3/records_selected' created successfully


/tmp/ipykernel_500/2718808716.py:109: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df_grouped:



Viso pacientų: 102  EKG įrašų: 1087

Atrinktų įrašų skaičiaus pasiskirstymas per pacientus (surūšiuotas)
userId                      Įrašų skaičius
61632bd41e32554ace017bc0    137
6144c682bd0cc5acb7275368    136
6144c588bd0cc52ba0275362     65
61632afa1e325516b4017bbb     57
6144c4fbbd0cc552e427535f     41
61632ace1e32557a62017bba     34
613b1d673d08d4d1f3cdc8f8     29
617509a66d4fb46d3b637744     28
6190d4b23cd1d29db2303ce9     25
613b1d0c3d08d413ffcdc8f6     25
61b320c5cf0f342a0acb1c95     25
619b4b92b4813271534e3da8     22
61b320a0cf0f344d5acb1c94     20
61b31ff1cf0f34275ccb1c90     20
61b320e9cf0f344a0fcb1c96     20
61a886195cd5477be0a4e96f     19
613b1c6f3d08d4370acdc8f3     18
61a8863e5cd547e980a4e970     18
61fe6753cd448a4569134712     17
6190d4e63cd1d227c1303cea     17
61cabfa9ec27507e7e60a4fe     16
61fe667ccd448a855313470d     15
61f167fa632a338a460c7a23     14
615708ba2312e518c0168e69     13
61f1676d632a33abc40c7a21     13
61fe66ffcd448a4cef134710     12
6190d45b3cd1d2e6953

/tmp/ipykernel_500/2718808716.py:163: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_sum = df_list_selected.groupby('userId').sum()
